# 보험 다모아 사이트 크롤링
- http://www.e-insmarket.or.kr/guaranteeIns/guaranteeInsIntro.knia
- 보장성 보험 데이터 크롤링
    - 암보험
- selenium 이용

In [1]:
from selenium import webdriver

In [2]:
# 드라이버 열기
driver = webdriver.Chrome()
driver.get("http://www.e-insmarket.or.kr/cancerIns/cancerInsList.knia")

In [3]:
# 년도 선택
driver.find_element_by_css_selector("#year [value='1989']").click()

In [4]:
# 월 선택
driver.find_element_by_css_selector("#month [value='1']").click()

In [5]:
# 일 선택
driver.find_element_by_css_selector("#day [value='1']").click()

In [6]:
# 상품 비교하기 버튼 클릭
driver.find_element_by_css_selector(".btn_type04").click()

In [7]:
# 엘리먼트들 선택하기
elements = driver.find_elements_by_css_selector(".table_type01 > tbody > .div")
len(elements)

51

In [8]:
# 각 요소 데이터 가져오기
items = []
for element in elements:
    item = {}
    try:
        item["company"] = element.find_element_by_css_selector(".comp_logo").text
        item["product"] = element.find_element_by_css_selector("td:nth-child(3)").text
        item["total_cost"] = element.find_element_by_css_selector(".total_cost").text
        item["range"] = element.find_element_by_css_selector("td:nth-child(7)").text
        item["age"] = element.find_element_by_css_selector("td:nth-child(8)").text
        item["etc"] = element.find_element_by_css_selector("td:nth-child(9)").text.replace("\n", ", ")     
        items.append(item)
        idx = 0
    except:
        guarantee = element.find_element_by_css_selector(".item_list > li")
        tmp_item = {}
        tmp_item["guarantee_" + str(idx)] = guarantee.find_element_by_css_selector(".kind").text
        tmp_item["guarantee_" + str(idx) + "_cost"] = guarantee.find_element_by_css_selector(".cost").text
        items[len(items)-1].update(tmp_item)
        idx += 1

In [9]:
# 브라우져 종료
driver.quit()

In [10]:
# 크롤링한 데이터 가져오기
items[:2]

[{'company': 'KDB생명',
  'product': '(무)KDB다이렉트 암보험(갱신형)',
  'total_cost': '1,350',
  'range': '94.2',
  'age': '0~60',
  'etc': '100세까지 보장, 암 진단시 최고 1억 보장',
  'guarantee_0': '일반암',
  'guarantee_0_cost': '1000만원',
  'guarantee_1': '유방암 및 남녀생식기암',
  'guarantee_1_cost': '200만원',
  'guarantee_2': '소액암',
  'guarantee_2_cost': '100만원'},
 {'company': '라이나생명',
  'product': '무배당 라이나다이렉트암보험(갱신형)',
  'total_cost': '1,746',
  'range': '88.4',
  'age': '20~50',
  'etc': '보장(담보)개수 6개, 10년만기, 10년납, 순수보장형',
  'guarantee_0': '암(유방암/전립선암)치료급여금',
  'guarantee_0_cost': '200만원',
  'guarantee_1': '갑상선암치료급여금',
  'guarantee_1_cost': '100만원',
  'guarantee_2': '기타피부암치료급여금',
  'guarantee_2_cost': '100만원',
  'guarantee_3': '제자리암(상피내암)치료급여금',
  'guarantee_3_cost': '100만원'}]

In [11]:
# 데이터 프레임 만들기
df = pd.DataFrame(items)
colums = ["company", "age", "product", "total_cost", "etc"]
df[colums].tail(2)

company    age                         product total_cost  \
13  미래에셋생명   0~55  미래에셋생명 온라인 암보험 무배당 1904[기본환급형]     11,780   
14    동양생명  19~60        (무)수호천사온라인꽉채운암보험(무해지환급형)     12,550   

                                etc  
13    보장(담보) 총5종, 80세만기 20년납, 순수보장형  
14  보장(담보) 총7종, 80세만기/20년납(월납)<....

### 함수로 만들기

In [12]:
def cancer_ins():
    
    driver = webdriver.Chrome()
    driver.get("http://www.e-insmarket.or.kr/cancerIns/cancerInsList.knia")
    
    driver.find_element_by_css_selector("#year [value='1989']").click()
    driver.find_element_by_css_selector("#month [value='1']").click()
    driver.find_element_by_css_selector("#day [value='1']").click()
    driver.find_element_by_css_selector(".btn_type04").click()
    
    items = []
    elements = driver.find_elements_by_css_selector(".table_type01 > tbody > .div")
    for element in elements:
        item = {}
        try:
            item["company"] = element.find_element_by_css_selector(".comp_logo").text
            item["product"] = element.find_element_by_css_selector("td:nth-child(3)").text
            item["total_cost"] = element.find_element_by_css_selector(".total_cost").text
            item["range"] = element.find_element_by_css_selector("td:nth-child(7)").text
            item["age"] = element.find_element_by_css_selector("td:nth-child(8)").text
            item["etc"] = element.find_element_by_css_selector("td:nth-child(9)").text.replace("\n", ", ")     
            items.append(item)
            idx = 0
        except:
            guarantee = element.find_element_by_css_selector(".item_list > li")
            tmp_item = {}
            tmp_item["guarantee_" + str(idx)] = guarantee.find_element_by_css_selector(".kind").text
            tmp_item["guarantee_" + str(idx) + "_cost"] = guarantee.find_element_by_css_selector(".cost").text
            items[len(items)-1].update(tmp_item)
            idx += 1
            
    driver.quit()
    
    return pd.DataFrame(items)

In [13]:
df = cancer_ins()
colums = ["company", "age", "product", "total_cost", "etc"]
df[colums].tail(2)

company    age                         product total_cost  \
13  미래에셋생명   0~55  미래에셋생명 온라인 암보험 무배당 1904[기본환급형]     11,780   
14    동양생명  19~60        (무)수호천사온라인꽉채운암보험(무해지환급형)     12,550   

                                etc  
13    보장(담보) 총5종, 80세만기 20년납, 순수보장형  
14  보장(담보) 총7종, 80세만기/20년납(월납)<....

In [14]:
df["name"] = "암보험"
colums.append("name")
result_df = df[colums]
result_df.tail(2)

company    age                         product total_cost  \
13  미래에셋생명   0~55  미래에셋생명 온라인 암보험 무배당 1904[기본환급형]     11,780   
14    동양생명  19~60        (무)수호천사온라인꽉채운암보험(무해지환급형)     12,550   

                                etc name  
13    보장(담보) 총5종, 80세만기 20년납, 순수보장형  암보험  
14  보장(담보) 총7종, 80세만기/20년납(월납)<....  암보험

In [15]:
# requests로 크롤링한 csv 파일 로드
load_df = pd.read_csv("guaranteeIns.csv")
load_df.tail(2)

name company         product total_cost    age  \
117  골프보험    KB손보  KB골프보험(다이렉트플랜)     29,260  20~69   
118  골프보험    롯데손보           홀인원보험     91,260  0~100   

                                  etc  
117  보장(담보) 총3종/1년 단기소멸성/ 기본형/표준형/고급형  
118          1년단기보험 (소멸성), 실속형/고급형 플랜

In [16]:
# 로드한 데이터 프레임과 크롤링한 데이터 프레임 합치기
result = load_df.append(result_df, ignore_index=True, sort=True)
result.tail()

age company                              etc name  \
129  19~50    신한생명      보장(담보) 총4종/80세만기/20년납/순수보장형  암보험   
130   0~55  미래에셋생명    보장(담보) 총5종, 80세만기 20년납, 순수보장형  암보험   
131  19~50  농협손해보험                  암진단비 80세만기/20년납  암보험   
132   0~55  미래에셋생명    보장(담보) 총5종, 80세만기 20년납, 순수보장형  암보험   
133  19~60    동양생명  보장(담보) 총7종, 80세만기/20년납(월납)<....  암보험   

                             product total_cost  
129              신한인터넷암보험(무배당)(비갱신형)     10,200  
130  미래에셋생명 온라인 암보험 무배당 1904[무해지환급형]     10,250  
131          (무)헤아림다이렉트암보험1904(비갱신형)     11,160  
132   미래에셋생명 온라인 암보험 무배당 1904[기본환급형]     11,780  
133         (무)수호천사온라인꽉채운암보험(무해지환급형)     12,550

In [17]:
# 파일 저장
result.to_csv("guaranteeIns.csv", index=False)